In [1]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets
from torchvision import transforms

import numpy as np

/home/jfialho/.conda/envs/ml_default/lib/python3.7/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [11]:
DEVICE = torch.device("cpu")
BATCHSIZE = 128
CLASSES = 10
DIR = os.getcwd()
EPOCHS = 1000
LOG_INTERVAL = 10
N_TRAIN_EXAMPLES = BATCHSIZE * 30
N_VALID_EXAMPLES = BATCHSIZE * 10

In [ ]:
dropout_l0 = 0.2083603371473296
dropout_l1 = 0.29475991600696116
lr = 0.004143220502481869
n_layers = 2
n_units_l0 = 119
n_units_l1 = 89
optimizer = Adam

In [12]:
# hyperparameters set
optimizer_name = "Adam"
lr = 0.004143220502481869
n_layers = 2
p = 0.29475991600696116
out_features = 89

In [ ]:
layers = []
acc = []

# Generate the model.
in_features = 28 * 28
for i in range(n_layers):        
    layers.append(nn.Linear(in_features, out_features))
    layers.append(nn.ReLU())
    layers.append(nn.Dropout(p))
    in_features = out_features
layers.append(nn.Linear(in_features, CLASSES))
layers.append(nn.LogSoftmax(dim=1))

model =  nn.Sequential(*layers).to(DEVICE)


# Generate the optimizers.
optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

# Load FashionMNIST dataset.
train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST(DIR, train=True, download=True, transform=transforms.ToTensor()),
    batch_size=BATCHSIZE,
    shuffle=True,
)
valid_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST(DIR, train=False, transform=transforms.ToTensor()),
    batch_size=BATCHSIZE,
    shuffle=True,
)


# Training of the model.
for epoch in range(EPOCHS):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # Limiting training data for faster epochs.
        if batch_idx * BATCHSIZE >= N_TRAIN_EXAMPLES:
            break

        data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)

        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

    # Validation of the model.
    model.eval()
    correct = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(valid_loader):
            # Limiting validation data.
            if batch_idx * BATCHSIZE >= N_VALID_EXAMPLES:
                break
            data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
            output = model(data)
            # Get the index of the max log-probability.
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    accuracy = correct / min(len(valid_loader.dataset), N_VALID_EXAMPLES)
    acc.append(accuracy)

    print("epoch:",epoch,"| acc:",accuracy)
    
#print best value of acc    
print("--------------------------")
print("best acc:", np.max(acc))

epoch: 0 | acc: 0.66640625
epoch: 1 | acc: 0.71015625
epoch: 2 | acc: 0.7546875
epoch: 3 | acc: 0.79296875
epoch: 4 | acc: 0.7796875
epoch: 5 | acc: 0.81484375
epoch: 6 | acc: 0.79921875
epoch: 7 | acc: 0.81171875
epoch: 8 | acc: 0.83828125
epoch: 9 | acc: 0.821875
epoch: 10 | acc: 0.8296875
epoch: 11 | acc: 0.81328125
epoch: 12 | acc: 0.81328125
epoch: 13 | acc: 0.82265625
epoch: 14 | acc: 0.82421875
epoch: 15 | acc: 0.8140625
epoch: 16 | acc: 0.8359375
epoch: 17 | acc: 0.83671875
epoch: 18 | acc: 0.840625
epoch: 19 | acc: 0.83828125
epoch: 20 | acc: 0.83359375
epoch: 21 | acc: 0.8234375
epoch: 22 | acc: 0.83125
epoch: 23 | acc: 0.85390625
epoch: 24 | acc: 0.84921875
epoch: 25 | acc: 0.83203125
epoch: 26 | acc: 0.83359375
epoch: 27 | acc: 0.82890625
epoch: 28 | acc: 0.83828125
epoch: 29 | acc: 0.84921875
epoch: 30 | acc: 0.8421875
epoch: 31 | acc: 0.83671875
epoch: 32 | acc: 0.83046875
epoch: 33 | acc: 0.8328125
epoch: 34 | acc: 0.84296875
epoch: 35 | acc: 0.8359375
epoch: 36 | acc: 0

In [ ]:
import matplotlib.pyplot as plt
plt.plot(acc)
plt.plot()